In [1]:
import os
import sys
import redis
from dotenv import load_dotenv

load_dotenv()

redis_host = os.getenv("REDIS_HOST") or "localhost"
redis_port = os.getenv("REDIS_PORT") or 6379


In [2]:
from redis.commands.json.path import Path

client = redis.Redis(host=redis_host, port=redis_port, db=0)

jane = {
     'name': "Jane",
     'Age': 33,
     'Location': "Chawton"
   }

client.json().set('person:1', '$', jane)

result = client.json().get('person:1')
print(result)

{'name': 'Jane', 'Age': 33, 'Location': 'Chawton'}


In [3]:
jane = {
     'name': "Jane",
     'Age': 33,
     'Location': "Chawton"
   }

client.json().set('person:1-session:1234', '$', jane)

result = client.json().get('person:1-session:*')
print(result)

None


In [4]:
jane = {
     'name': "Jane",
     'sessions': []
   }

session = {
      'session_id': "1234",
      'role': "user",
      'content': "Hello"
}

session2 = {
      'session_id': "1234-2",
      'role': "user",
      'content': "Hello 2"
}

client.json().set('person:1', '$', jane)
client.json().arrappend('person:1', '.sessions', session)
client.json().arrappend('person:1', '.sessions', session2)

result = client.json().get('person:1')
print(result)

{'name': 'Jane', 'sessions': [{'session_id': '1234', 'role': 'user', 'content': 'Hello'}, {'session_id': '1234-2', 'role': 'user', 'content': 'Hello 2'}]}


ahora solo quiero la sesion 2

In [5]:
s2 = client.json().get('person:1', '.sessions[1]')
s2

{'session_id': '1234-2', 'role': 'user', 'content': 'Hello 2'}

quiero buscarla por el nombre de la sesion, no me sirve guardar todo junto. Le meto el person_id a la session y listo.

In [6]:
session_header = {
    'session_id': "10",
     'person_id': "10",
      'name': "Jane",
     'messages': []
   }

message1 = {
      'role': "user",
      'content': "Hello"
}
message2 = {
    'role': "system",
    'content': "Hello 2"
}

client.json().set('session:10', '$', session_header)
client.json().arrappend('session:10', '.messages', message1)
client.json().arrappend('session:10', '.messages', message2)

result = client.json().get('session:10')
print(result)

{'session_id': '10', 'person_id': '10', 'name': 'Jane', 'messages': [{'role': 'user', 'content': 'Hello'}, {'role': 'system', 'content': 'Hello 2'}]}


In [22]:
session_header = {
    'session_id': "20",
     'person_id': "10",
      'name': "Jane",
     'messages': []
   }

message1 = {
      'role': "user",
      'content': "holis"
}
message2 = {
    'role': "system",
    'content': "re holis 2"
}

client.json().set('session:20', '$', session_header)
client.json().arrappend('session:20', '.messages', message1)
client.json().arrappend('session:20', '.messages', message2)

result = client.json().get('session:1')
print(result)

{'session_id': '2', 'person_id': '1', 'name': 'Jane', 'messages': [{'role': 'user', 'content': 'holis'}, {'role': 'system', 'content': 're holis 2'}]}


tiene que hacer un scan para traer todas las sesiones de un person_id. no me gusta como solucion. Sí está bueno para guardar la sesión actual, pero cuántos segundos me ahorro?

voy a seguir con este esquema pero cuando termine la sesion la saco de redis y la mando a mongo. Mientras, los datos clave que vaya sacando el modelo se guardan en vectores. 

o tambien lo que puedo hacer es guardar toda la sesion en vectores con el person_id y session_id como metadata. Asi es sistema tiene data que fue recabando y ademas toda la historia de las conversaciones y si pasa algo similar lo puede usar para recomendar.

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [18]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [23]:
session10 = client.json().get('session:10')
session10


{'session_id': '10',
 'person_id': '10',
 'name': 'Jane',
 'messages': [{'role': 'user', 'content': 'Hello'},
  {'role': 'system', 'content': 'Hello 2'}]}

In [30]:
metadatas = [{
    'session_id': "10",
      'person_id': "10",
}]
# turn json to plain text
content = session10['messages']
str(content)

db.add_texts(texts=[str(content)], embedding=embeddings, metadatas=metadatas)

['38a12a1b-49eb-4c92-b7ec-26da0d4caa0f']

In [31]:
session20 = client.json().get('session:20')
metadatas = [{
    'session_id': "20",
      'person_id': "10",
}]
# turn json to plain text
content = session20['messages']
str(content)

db.add_texts(texts=[str(content)], embedding=embeddings, metadatas=metadatas)

['644524bb-bc2e-4795-8b3e-b09abb0087d1']

In [32]:
query = "cual es la segunda sesion?"
filter = {"person_id": "10"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=1)

[(Document(page_content="[{'role': 'user', 'content': 'holis'}, {'role': 'system', 'content': 're holis 2'}]", metadata={'session_id': '20', 'person_id': '10'}),
  0.6522268)]

In [33]:
query = "cual es la primera sesion?"
filter = {"person_id": "10"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=1)

[(Document(page_content="[{'role': 'user', 'content': 'holis'}, {'role': 'system', 'content': 're holis 2'}]", metadata={'session_id': '20', 'person_id': '10'}),
  0.65058786)]

No me entiende lo de primera, voy a meter fecha en formato semantico

In [34]:
import datetime
import locale

# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES')

timestamp = datetime.datetime.now()
formatted_timestamp = timestamp.strftime('%A, %d de %B de %Y, %H:%M:%S')

print(formatted_timestamp)

martes, 18 de julio de 2023, 11:10:15


In [49]:
session_header = {
    'session_id': "1000",
     'person_id': "1000",
      'name': "Jane",
     'messages': []
   }

formatted_timestamp = "lunes, 17 de julio de 2023, 14:30:00"

message1 = {
      'timestamp': formatted_timestamp,
      'role': "user",
      'content': "Hello"
}
formatted_timestamp = "lunes, 17 de julio de 2023, 14:31:00"

message2 = {
    'timestamp': formatted_timestamp,
    'role': "system",
    'content': "Hello 2"
}

client.json().set('session:1000', '$', session_header)
client.json().arrappend('session:1000', '.messages', message1)
client.json().arrappend('session:1000', '.messages', message2)

result = client.json().get('session:1000')
print(result)

{'session_id': '1000', 'person_id': '1000', 'name': 'Jane', 'messages': [{'timestamp': 'lunes, 17 de julio de 2023, 14:30:00', 'role': 'user', 'content': 'Hello'}, {'timestamp': 'lunes, 17 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2'}]}


In [50]:
session_header = {
    'session_id': "2000",
     'person_id': "1000",
      'name': "Jane",
     'messages': []
   }

formatted_timestamp = "martes, 18 de julio de 2023, 14:30:00"

message1 = {
      'timestamp': formatted_timestamp,
      'role': "user",
      'content': "Hello martes"
}
formatted_timestamp = "martes, 18 de julio de 2023, 14:31:00"

message2 = {
    'timestamp': formatted_timestamp,
    'role': "system",
    'content': "Hello 2 martes"
}

client.json().set('session:2000', '$', session_header)
client.json().arrappend('session:2000', '.messages', message1)
client.json().arrappend('session:2000', '.messages', message2)

result = client.json().get('session:2000')
print(result)

{'session_id': '2000', 'person_id': '1000', 'name': 'Jane', 'messages': [{'timestamp': 'martes, 18 de julio de 2023, 14:30:00', 'role': 'user', 'content': 'Hello martes'}, {'timestamp': 'martes, 18 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2 martes'}]}


In [52]:
session1000 = client.json().get('session:100')
metadatas = [{
    'session_id': "1000",
      'person_id': "1000",
}]
# turn json to plain text
content = session1000['messages']
str(content)

db.add_texts(texts=[str(content)], embedding=embeddings, metadatas=metadatas)

['5897d9d9-e8f8-4336-ae2f-1f0dc8cb2088']

In [53]:
session2000 = client.json().get('session:2000')
metadatas = [{
    'session_id': "2000",
      'person_id': "1000",
}]
# turn json to plain text
content = session2000['messages']
str(content)

db.add_texts(texts=[str(content)], embedding=embeddings, metadatas=metadatas)

['0abb3364-d8df-4a34-bc74-2786d52cbd29']

In [41]:
query = "cual es la primera sesion?"
filter = {"person_id": "100"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=1)

[(Document(page_content="[{'timestamp': 'martes, 17 de julio de 2023, 14:30:00', 'role': 'user', 'content': 'Hello martes'}, {'timestamp': 'martes, 17 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2 martes'}]", metadata={'session_id': '200', 'person_id': '100'}),
  0.56230164)]

In [57]:
query = "lunes, 17 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2'"

filter = {"person_id": "1000"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=3)


[(Document(page_content="[{'timestamp': 'lunes, 17 de julio de 2023, 14:30:00', 'role': 'user', 'content': 'Hello'}, {'timestamp': 'lunes, 17 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2'}]", metadata={'session_id': '1000', 'person_id': '1000'}),
  0.2698342),
 (Document(page_content="[{'timestamp': 'martes, 18 de julio de 2023, 14:30:00', 'role': 'user', 'content': 'Hello martes'}, {'timestamp': 'martes, 18 de julio de 2023, 14:31:00', 'role': 'system', 'content': 'Hello 2 martes'}]", metadata={'session_id': '2000', 'person_id': '1000'}),
  0.2793008)]